# 挑战 25：各国历年二氧化碳 CO2 排放量统计分析

## 1. 数据清洁

### 读取数据

In [1]:
import pandas as pd

# 读取数据文件
df_data = pd.read_excel("ClimateChange.xlsx", sheet_name='Data')
df_country = pd.read_excel("ClimateChange.xlsx", sheet_name='Country')

### 处理 data 数据表

In [2]:
# 选取 EN.ATM.CO2E.KT 数据，并将国家代码设置为索引

df_data_reindex = df_data[df_data['Series code']== 'EN.ATM.CO2E.KT'].set_index('Country code')

In [3]:
# 剔除不必要的数据列
df_data_drop = df_data_reindex.drop(labels=['Country name', 'Series code', 'Series name', 'SCALE', 'Decimals'], axis=1)

In [4]:
df_data_nan = df_data_drop.replace({'..': pd.np.NaN})

In [5]:
# 对 NaN 空值进行向前和向后填充
df_data_fill = df_data_nan.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)

In [6]:
# 对填充后依旧全部为空值的数据行进行剔除
df_data_dropna = df_data_fill.dropna(how='all')

### 处理 Country 数据表

In [7]:
# 将国家代码设置为索引
df_country_reindex = pd.DataFrame(df_country).set_index('Country code')

In [8]:
# 剔除不必要的数据列
df_country_drop = df_country_reindex.drop(labels=['Capital city', 'Region', 'Lending category'], axis=1)

### 合并数据表

In [9]:
# 对 Data 和 Country 表按照索引进行合并
df_combine = pd.concat([df_data_dropna, df_country_drop], axis=1, sort=True)

In [10]:
# 对合并后数据集进行求和得到各国排放总量
df_combine['Sum emissions'] = df_combine[list(df_combine)[:-2]].sum(axis=1)

In [11]:
# 对合并后存在空值的数据行进行剔除，得到清洁后的数据集
df_clean = df_combine.dropna(thresh=10)

## 2. 数据求和整理

### 按收入群体对数据进行求和

In [12]:
# 按收入群体对数据进行求和
sum_by_groups = df_clean.groupby('Income group')['Sum emissions'].sum()

### 按要求整理 DataFrame

In [13]:
# 按要求整理 DataFrame
item_high_list = []
item_low_list = []

for group_name in list(sum_by_groups.index):

    # 得到各收入群体最高排放量数据
    item_high = df_clean[df_clean['Income group'] == group_name].sort_values(
        by='Sum emissions', ascending=False).iloc[0]

    # 将最高排放量数据存入相应列表方便生成最终 DataFrame
    item_high_list.append(
        (item_high['Income group'], item_high['Country name'], item_high['Sum emissions']))

    # 得到各收入群体最低排放量数据
    item_low = df_clean[df_clean['Income group'] ==
                        group_name].sort_values(by='Sum emissions').iloc[0]

    # 将最低排放量数据存入相应列表方便生成最终 DataFrame
    item_low_list.append(
        (item_low['Income group'], item_low['Country name'], item_low['Sum emissions']))

### 合并输出

In [14]:
# 设置 DataFrame 标签
high_labels = ['Income group', 'Highest emission country', 'Highest emissions']
low_labels = ['Income group', 'Lowest emission country', 'Lowest emissions']

# 生成并合并目标 DataFrame
highest_df = pd.DataFrame.from_records(item_high_list, columns=high_labels).set_index('Income group')
lowest_df = pd.DataFrame.from_records(item_low_list, columns=low_labels).set_index('Income group')

results = pd.concat([sum_by_groups, highest_df, lowest_df], axis=1)
results

,Sum emissions,Highest emission country,Highest emissions,Lowest emission country,Lowest emissions
Income group,,,,,
High income: OECD,2.588373e+08,United States,1.179918e+08,Iceland,46944.934
High income: nonOECD,1.581112e+07,Saudi Arabia,7.009760e+06,Turks and Caicos Islands,1503.470
Low income,5.485641e+06,"Korea, Dem. Rep.",3.104479e+06,Comoros,2068.188
Lower middle income,6.272726e+07,India,2.681828e+07,Kiribati,601.388
Upper middle income,2.100775e+08,China,9.809777e+07,Niue,80.674


## 解法 2

从清洁后数据 `df_clean` 开始：

In [15]:
df_max = df_clean.sort_values(by='Sum emissions', ascending=False).drop_duplicates(
    'Income group').set_index('Income group')[['Country name', 'Sum emissions']]
df_max.columns = ['Highest emission country','Highest emissions']
df_max

,Highest emission country,Highest emissions
Income group,,
High income: OECD,United States,1.179918e+08
Upper middle income,China,9.809777e+07
Lower middle income,India,2.681828e+07
High income: nonOECD,Saudi Arabia,7.009760e+06
Low income,"Korea, Dem. Rep.",3.104479e+06


In [16]:
df_min = df_clean.sort_values(by='Sum emissions', ascending=True).drop_duplicates(
    'Income group').set_index('Income group')[['Country name', 'Sum emissions']]
df_min.columns = ['Lowest emission country', 'Lowest emissions']
df_min

,Lowest emission country,Lowest emissions
Income group,,
Upper middle income,Niue,80.674
Lower middle income,Kiribati,601.388
High income: nonOECD,Turks and Caicos Islands,1503.470
Low income,Comoros,2068.188
High income: OECD,Iceland,46944.934


In [17]:
pd.concat([sum_by_groups, df_max, df_min], sort=False, axis=1)

,Sum emissions,Highest emission country,Highest emissions,Lowest emission country,Lowest emissions
High income: OECD,2.588373e+08,United States,1.179918e+08,Iceland,46944.934
High income: nonOECD,1.581112e+07,Saudi Arabia,7.009760e+06,Turks and Caicos Islands,1503.470
Low income,5.485641e+06,"Korea, Dem. Rep.",3.104479e+06,Comoros,2068.188
Lower middle income,6.272726e+07,India,2.681828e+07,Kiribati,601.388
Upper middle income,2.100775e+08,China,9.809777e+07,Niue,80.674
